In [1]:
import redis
import h3
import numpy as np

In [2]:
r = redis.Redis('localhost') # make sure this is running, port default is 6379?

In [4]:
scan = r.scan('',count=2000)
print(len(scan[1]))
print(scan[1])

0
[]


In [13]:
res = 7
h3.geo_to_h3(-32,4,res)

'87ad4536cffffff'

In [14]:
r.get('87ad4536cffffff')

b''

In [11]:
#find_jobs((0,0.01))

add_job(1, (0, 0))

87754e64dffffff


8

To do:
- kl coords as testing data
- haversine ground truth
- helper functions (in notebook) -> redis and h3
- helper functions (via import)
- API:
- add_job endpoint
- nearby_jobs (coords, radius = 5) [[VARIABLE PRECISION??]] (do this tmr?)


---

check job??

In [5]:
min_lat, max_lat = 3.022048, 3.225460
min_lng, max_lng = 101.509419, 101.760425

h3.geo_to_h3(min_lat, min_lng, res)

'8765050c0ffffff'

In [6]:
import numpy as np
def haversine(s_lat, s_lng, e_lat, e_lng):
    # approximate radius of earth in km
    R = 6373.0

    s_lat = np.deg2rad(s_lat)
    s_lng = np.deg2rad(s_lng)
    e_lat = np.deg2rad(e_lat)
    e_lng = np.deg2rad(e_lng)

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))

haversine( min_lat, min_lng, min_lat, max_lng )

27.880528607626378

In [36]:
len(sets[0])

19

In [37]:
## redis funcs
for h in hashes:
    print(r.get(h))

b'0 '
b'1 '
b'2 12 44 '
b'3 38 '
b'4 '
b'5 42 '
b'6 32 '
b'7 '
b'8 '
b'9 34 '
b'10 '
b'11 35 '
b'2 12 44 '
b'13 '
b'14 '
b'15 '
b'16 '
b'17 '
b'18 '
b'19 '
b'20 28 '
b'21 47 48 '
b'22 26 '
b'23 '
b'24 '
b'25 '
b'22 26 '
b'27 '
b'20 28 '
b'29 '
b'30 '
b'31 '
b'6 32 '
b'33 '
b'9 34 '
b'11 35 '
b'36 '
b'37 '
b'3 38 '
b'39 '
b'40 '
b'41 '
b'5 42 '
b'43 '
b'2 12 44 '
b'45 '
b'46 '
b'21 47 48 '
b'21 47 48 '
b'49 '


In [8]:
# set r = Redis.redis()

h3_resolution = 7

def add_job(id_, coords):
    hash_ = h3.geo_to_h3(*coords, h3_resolution)
    print(hash_)
    return r.append(hash_, f'{id_} ')

def find_jobs(coords, radius = 5):
    ## RADIUS CALC for later
    hash_ = h3.geo_to_h3(*coords, h3_resolution)
    str_list = r.mget( h3.k_ring(hash_, 2) ) 
    ids = b''.join( (x for x in str_list if x ) ).split(b' ') #above line always returns bytes from redis
    ids = [int(x) for x in ids if x!= b'']
    return ids

# get old hash
# split by ' '
# look for id 
# remove that element
# rebuild string

script = """ 
--KEYS: [id, old hash, new hash]
--SEARCHES AND DELETES OLD ID FROM old hash, THEN APPENDS IT INTO new hash
local deletion_str = KEYS[1] .. ' '
local old_hash_str = redis.call('get', KEYS[2])

local replacement = string.gsub( old_hash_str, "%w+ ", function (s)
if s == deletion_str then return '' else return s end
end )

local set_resp = redis.call('set', KEYS[2], replacement )
local append_resp = redis.call('append', KEYS[3], deletion_str)

return append_resp

"""
edit_cmd =  r.register_script(script) ## keys: [id, old hash, new hash]

def edit_job_loc(id_, old_coords, new_coords):
    #redis transaction that takes the old_coords hash, cuts out the id from that string, then puts it into the new_coords key
    # !warning! will fail to remove if old_coords is not the right value, doesn't verify this.
    old_hash = h3.geo_to_h3(*old_coords, h3_resolution)
    new_hash = h3.geo_to_h3(*new_coords, h3_resolution)
    
    return edit_cmd([id_, old_hash, new_hash])
    


In [39]:
add_job(3, (0,0))

87754e64dffffff


2

In [40]:
print(r.get('87754e64dffffff'))

b'3 '


In [41]:
find_jobs((0,0))

[3]

## Testing

In [45]:
r.flushall()

True

In [125]:
samples = 1000
random_lats = np.random.uniform(min_lat,max_lat, size=samples)
random_lngs = np.random.uniform(min_lng,max_lng, size=samples)
coords = np.vstack([random_lats, random_lngs]).T
# coords[i] -> random_lats[i], random_lngs[i]

In [126]:
distances = []
for i in range(samples):
    distances.append( haversine( random_lats[i], random_lngs[i] , random_lats , random_lngs) )

In [127]:
np.set_printoptions(precision=2 , linewidth = 200, suppress=True)
distances = np.array(distances)
distances.shape, distances

((5000, 5000),
 array([[ 0.  , 10.69, 12.31, ..., 22.93, 18.93, 14.9 ],
        [10.69,  0.  ,  3.86, ..., 12.25, 10.12,  4.28],
        [12.31,  3.86,  0.  , ..., 11.55,  6.83,  4.23],
        ...,
        [22.93, 12.25, 11.55, ...,  0.  ,  8.84,  8.04],
        [18.93, 10.12,  6.83, ...,  8.84,  0.  ,  7.2 ],
        [14.9 ,  4.28,  4.23, ...,  8.04,  7.2 ,  0.  ]]))

In [49]:
distances < 5

array([[ True, False, False, ..., False,  True, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [ True, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False,  True]])

In [128]:
## h3 funcs
hashes = []
sets = []
for i in range(samples):
    h = h3.geo_to_h3(random_lats[i],random_lngs[i], 7) 
    set_ = h3.k_ring(h, 2)
    hashes.append(h)
    r.append(h,f"{i} ")
    sets.append(set_)

#print(hashes) 
#print(sets)   

In [116]:
def calculate_confusion_matrix(target, predict, classes_num):
    """Calculate confusion matrix.

    Inputs:
      target: integer array, (audios_num,)
      predict: integer array, (audios_num,)
      classes_num: int, number of classes

    Outputs:
      confusion_matrix: (classes_num, classes_num)
    """

    confusion_matrix = np.zeros((classes_num, classes_num))
    samples_num = len(target)

    for n in range(samples_num):
        confusion_matrix[target[n], predict[n]] += 1

    return confusion_matrix 

def calculate_confusion_matrix_from_arrays(prediction, ground_truth, nr_labels):
    """
    calculate the confusion matrix for one image pair.
    prediction and ground_truth have to have the same shape.
    """

    # a long 2xn array with each column being a pixel pair
    replace_indices = np.vstack((
        ground_truth.flatten(),
        prediction.flatten())
    ).T

    # add up confusion matrix
    confusion_matrix, _ = np.histogramdd(
        replace_indices,
        bins=(nr_labels, nr_labels),
        range=[(0, nr_labels), (0, nr_labels)]
    )
    confusion_matrix = confusion_matrix.astype(np.uint)
    return confusion_matrix 

In [129]:
#distances[i] < 5

#c_m = np.zeros((2,2), dtype=np.uint)

for i in range(samples):
    truths = (distances[i] < 5).astype(np.uint)
    according_to_h3 = np.array([(h in sets[i])  for h in hashes], dtype=np.uint)
    c_m += calculate_confusion_matrix(truths, according_to_h3, 2)

    
c_m[1,1] -= samples # the diagonal compares a location vs itself. Exclude this 
    
c_m



array([[22161979.,  1110979.],
       [   35994.,  2686048.]])

In [131]:
rec = c_m[1,1]/(c_m[1,0] + c_m[1,1])
pre = c_m[0,0]/(c_m[0,0] + c_m[0,1])

print(pre, rec)

0.9522630943604161 0.9867768388584747


In [134]:
from fastapi import FastAPI, HTTPException, Depends, Request, Header
import time
import hashlib

app = FastAPI()


@app.post("/add_job")
async def add_job_service(id: int, lat: float, lng: float, authorized: bool = Depends(verify_user)):
    resp = add_job(id, (lat,lng))
    return {"message": resp}

@app.post("/find_jobs")
async def find_job_service(lat: float, lng: float, authorized: bool = Depends(verify_user)):
    resp = find_jobs(id, (lat,lng))
    return {"job_ids": resp}

secret = 'xd'
salt_cache = cachetools.TTLCache(1000000, 5)

def verify_user(req: Request):
    try:
        time_token = req.headers["time-token"]
        h = req.headers["auth-key"]
    except KeyError:
        raise HTTPException(status_code=400, detail="Bad Request", message="headers missing")
    
    if req.json['lat'] == 0:
        return True 
    
    #no reusing old tokens
    if not (time.time_ns() - 5 * 1000000000000 < time_token < time.time_ns()): 
        raise HTTPException(status_code=401, detail="Unauthorized")
        
    #no using tokens more than once
    if salt_cache.get(time_token, None):
        raise HTTPException(status_code=401, detail="Unauthorized")
    
    #test integrity of hash
    valid_h = hashlib.new('sha512')
    valid_h.update(f'{time_token}-{secret}'.encode())
    
    if h != valid_h.hexdigest():
        raise HTTPException(status_code=401, detail="Unauthorized")
        
    else: 
        salt_cache[time_token] = True #no using tokens more than once
        return True
    
@app.get("/")
async def home(authorized: bool = Depends(verify_user)):
     if authorized:
        return {"detail": "Welcome home"}



4

TO DO TN:
- LUA SCRIPT
- what loose ends do i need to settle tmr??
- plan for tmr
- update team

In [258]:
r.scan('')
r.get('876505096ffffff')

b'3 9 '

In [4]:
r.set('test:876505096ffffff', '10 45 ')

True

In [9]:
r.get('test:876505096ffffff')

b'10 45 '

In [19]:
#edit_job_loc(3, (0,0), (2,2))

In [136]:
!pip3 install cachetools

In [1]:
!pip3 install python-dotenv

In [179]:
##FIX THE REQS later...... 

!pip3 install uvicorn[standard]

     |████████████████████████████████| 50 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.8 MB/s  eta 0:00:01
  Using cached asgiref-3.3.4-py3-none-any.whl (22 kB)
     |████████████████████████████████| 97 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 4.7 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 354 kB 11.5 MB/s eta 0:00:01


In [132]:
import sqlite3

## Authorization method:

- use secret key.
- put the timestamp in the header, it doesnt matter if outsiders can read it, its only used as salt, 
- append it to private key
- hash
- then every request you check that the timestamp is only a couple seconds late, and if the hash matches
- can use a cache to also invalidate those timestamps as soon as they come in, so that no one can borrow it for another use

In [137]:
import cachetools

In [168]:
c = cachetools.TTLCache(1, 5)
c.setdefault(None)

In [177]:
c[1] = 'done'
c[2] = 'done'

In [179]:
print(c.get(1, None))
print(c.get(2, None))

None
None


False

In [185]:
import hashlib
h = hashlib.new('sha512')
h.update(b'yooooo what up')
h.hexdigest()

'86b40d8c50cfdaf80d54a6bb313dae5674223637a8649e7d9c4acf1fe0e26db695a0805521b04a3cc9c7998f10b3ce7d019c9f1f00013ace000b478d16090c5e'

In [196]:
#from helper import hi
import helper

In [194]:
helper = None

In [212]:
class lol:
    def __init__(self):
        self.lol = 'lol'
        self.s = self.script
        
    script = 'scr'

In [213]:
lol().script

'scr'

In [2]:
questions = [
    {
        "question": "Co-operatives are better than traditional business models.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should pursue equality of outcome.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Land should not be taxed.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The hiring and firing of workers should not be down to the whims of bosses.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Private schools should be abolished.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "Monopolies, including state monopolies, should be avoided at all costs.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Inheritance and estate taxes are unethical.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "Some level of inequality is necessary for society to function.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Productivity is not as important as providing jobs.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should consider implementing a land value tax.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "People should be economically deterred from spending their money on useless products.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "There should be no central bank.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Tax money should not fund the lifestyles of monarchs or aristocrats.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "A society without rulers is desirable.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should attempt to consume local products as much as possible.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The government and big businesses collude to stomp out small scale enterprises.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Union membership should never be part of an employment contract.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "The land belongs to the people.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The minimum wage should be abolished.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Workers should negotiate for wages independently not collectively.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "Workers should not be given too much autonomy or they’ll be lazy.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "All workplaces should be unionized.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "Slave labor is justified if its productive enough.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Price gouging during shortages is exploitative.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Repetitive work should be done by machines.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Hand made products are usually of a better quality than machine made alternatives.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Without property rights, there are no rights at all.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "If capitalism exists at all, it’s worst effects should be mitigated by welfare.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "A general strike could be used to improve society significantly.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "Some people are better suited to make economic decisions than others.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Rent is a form of theft.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Collective bargaining reduces the autonomy of an individual worker.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "Companies should not hire or fire people based on factors like race or gender.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "Humans will always want to work.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "There should be no managers with authority over workers.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should intervene to reduce the production of harmful goods.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The economy should be centrally planned.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Consumerism has had a positive effect on modern economies.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Environmental regulations should be abolished.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Decentralised economies are inefficient.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Large scale organisations can increase efficiency.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Gambling should be legal.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Profits made by a company should be shared amongst its employees.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Strikes should be used to achieve better working conditions.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "When someone dies, their possessions and wealth rightfully belong to their next of kin.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "We should aim to achieve an automated economy as fast as possible.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Bosses usually know what’s best for their workers.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Education should be free and universal.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "An individual’s role in society should be inherited from their parents.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "The right to own property is absolute.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Price fixing will lead to shortages.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "A flat tax is preferable to a progressive tax.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Some jobs should be reserved for people born into higher class families.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "We must make sure everyone has enough to live, even if that negatively affects the economy.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Unions are not the solution to economic problems and can often make things worse.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "Healthcare should never be free.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Independent labor unions should not be allowed to exist.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "Workers should come together to form one big union.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "People who choose not to work should not expect support from society.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Prices should be determined by supply and demand.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "If someone is first to claim an area of land, they are entitled to profit from it.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Since machines work at a lower cost than humans, we should embrace them.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Taxes should be raised on the rich to provide for the poor.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should pursue equality of opportunity.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "Nobody should profit from natural resources which they didn’t earn.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Certain services like roads and public transportation should be run by the state.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Top down structures for organisations work best.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Unemployment caused by technology can be a good thing, as people will have to work fewer hours.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Landlords provide an essential service.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "People have a right to occupy and use unused land, even if it’s owned by someone.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "In the future, all the work people don’t want to do will be mechanised.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The economy should be organised on the local level.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "We should try to achieve full employment.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Universal basic income is a good idea.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "There should be rules to prevent the common people acquiring wealth.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "We should direct the economy away from the production of luxuries.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Machines should be destroyed to prevent workers losing their jobs.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "What is produced should not be down to the whims of the market.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Organised labor should be used as the primary vehicle for political change.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "Government run industry is superior to existing alternatives.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Work makes people stronger.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Employment should be guaranteed.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": -1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Charities only help the weak and should be eliminated.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "People should not be allowed to trade human organs.",
        "effect": {
            "bince": 1,
            "bstru": 0,
            "binte": -1,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Intervention in the market is permissible to correct market failures.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "The freer the market, the freer the people.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 1,
            "bcent": -1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "A central AI should determine the allocation of resources.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 1,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Cryptocurrencies are superior to fiat currency.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": 1,
            "bland": 0,
            "binhe": 0,
            "blabo": 0,
        }
    },
    {
        "question": "Workers should have more of a say in the workplace.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "More people should aim to be self employed.",
        "effect": {
            "bince": 0,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "Anyone should have the opportunity to gain a position of authority.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "A stratified class system is desirable.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 0,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "Inherited estate should be the basis for economic relations.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "Landlords can be good when they allow people to save up to buy their own home.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": -1,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "Tenant unions are a good idea.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "Common ownership of land is preferable to common ownership of capital.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": -1,
        }
    },
    {
        "question": "We should return to a feudal society.",
        "effect": {
            "bince": 0,
            "bstru": 0,
            "binte": 0,
            "bcent": -1,
            "btech": -1,
            "bland": -1,
            "binhe": 1,
            "blabo": 0,
        }
    },
    {
        "question": "We should attempt to establish socialism.",
        "effect": {
            "bince": 1,
            "bstru": 1,
            "binte": 0,
            "bcent": 0,
            "btech": 0,
            "bland": 1,
            "binhe": 0,
            "blabo": 1,
        }
    },
    {
        "question": "Capitalism has massively improved the world.",
        "effect": {
            "bince": -1,
            "bstru": 0,
            "binte": 1,
            "bcent": 0,
            "btech": 1,
            "bland": 0,
            "binhe": -1,
            "blabo": 0,
        }
    },
    {
        "question": "The state should control all aspects of the economy.",
        "effect": {
            "bince": 0,
            "bstru": -1,
            "binte": -1,
            "bcent": 1,
            "btech": 0,
            "bland": 0,
            "binhe": 0,
            "blabo": -1,
        }
    }
]

In [5]:
[print(q['question']) for q in questions] 

Co-operatives are better than traditional business models.
We should pursue equality of outcome.
Land should not be taxed.
The hiring and firing of workers should not be down to the whims of bosses.
Private schools should be abolished.
Monopolies, including state monopolies, should be avoided at all costs.
Inheritance and estate taxes are unethical.
Some level of inequality is necessary for society to function.
Productivity is not as important as providing jobs.
We should consider implementing a land value tax.
People should be economically deterred from spending their money on useless products.
There should be no central bank.
Tax money should not fund the lifestyles of monarchs or aristocrats.
A society without rulers is desirable.
We should attempt to consume local products as much as possible.
The government and big businesses collude to stomp out small scale enterprises.
Union membership should never be part of an employment contract.
The land belongs to the people.
The minimum wa

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]